In [3]:
#########CMSC-516########
#########NLP Project: SemEvalTask 2022########
#########To identify Sarcasm on provided tweets by Silviu & Team#########
##########Silviu is the organiser for this task iSarcasm#########
#########Credits: Huggingface & Vincent###########
##########Github Link for both Train and Test data are below#########
##########https://github.com/silviu-oprea/iSarcasm###########
!pip install transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback

In [4]:
######We are reading the training data#####
data = pd.read_csv("train.En.csv")

In [5]:
data[data.isna()["tweet"]]

,Unnamed: 0,tweet,sarcastic


In [6]:
######We are using BERT pretrained model(uncased) and takenizer#####
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [45]:
#####80:20 split#####
#####preprocessing the training set#######
#####input data with same len to the model with max 512########
X = list(data["tweet"])
y = list(data["sarcastic"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)
print("Total Length of the Train.csv File received:",len(X))
print("Sarcastic Tweets out of 3467 is:",data.sarcastic.sum())
print("Length of the Training Data:",len(X_train))
print("Length of the Validation Data:",len(X_val))
#y_train

Total Length of the Train.csv File received: 3467
Sarcastic Tweets out of 3467 is: 867
Length of the Training Data: 2773
Length of the Validation Data: 694


In [8]:
#########We now create the dataset in torch class as per the trainer API#########
##########To reuse the model on test data prediction we make labels=none######
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels
#######Now creating batches of data########
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item
###########length###########
    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [9]:
##########Defining our Evaluation metrics#######
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [32]:
#############We are defining the hyperparameters #########
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    seed=0,
    load_best_model_at_end=True,
)
##########Defining the trainer function########
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [33]:
#########Training the trainer model#########
#########Fine tuning BERT model##############
trainer.train()

***** Running training *****
  Num examples = 2773
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2082


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.392300,1.153550,0.759366,0.533333,0.232558,0.323887
1000,0.243400,1.173473,0.744957,0.471910,0.244186,0.321839
1500,0.047800,1.779415,0.737752,0.445652,0.238372,0.310606
2000,0.015800,2.000980,0.729107,0.424528,0.261628,0.323741


***** Running Evaluation *****
  Num examples = 694
  Batch size = 8
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 694
  Batch size = 8
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 694
  Batch size = 8
Saving model checkpoint to output/checkpoint-1500
Configuration saved in output/checkpoint-1500/config.json
Model weights saved in output/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 694
  Batch size = 8
Saving model checkpoint to output/checkpoint-2000
Configuration saved in output/checkpoint-2000/config.json
Model weights saved in output/checkpoint-2000/pytorch_model.bin


Training completed. Do not forget to 

TrainOutput(global_step=2000, training_loss=0.17479741978645325, metrics={'train_runtime': 1085.9116, 'train_samples_per_second': 15.322, 'train_steps_per_second': 1.917, 'total_flos': 952883409202800.0, 'train_loss': 0.17479741978645325, 'epoch': 5.76})

In [34]:
###########Now testing our test file aginst the pretrained model############
###########We will be repeating the same steps we did for train data for our test data as well############
###########Loading Test file#############

test_data = pd.read_csv("FinalTest.csv")
test_data.loc[(test_data['sarcastic'] == 'sarcastic'), 'sarcastic'] = 1
test_data.loc[(test_data['sarcastic'] == 'not_sarcastic'), 'sarcastic'] = 0
X_test = list(test_data["tweet"])
Y_test = list(test_data["sarcastic"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)


In [35]:
test_data

,Id,tweet,sarcastic,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,1081209061251710976,so nice not having to worry about finding a st...,1,NaN,NaN,NaN,NaN
1,1043020074670415872,I do love how #banking works- As a customer yo...,1,NaN,NaN,NaN,NaN
2,1126482905688100864,Sometimes I listen to the uga fight song and c...,1,NaN,NaN,NaN,NaN
3,1166817091489542144,"Our magical cup run has come to a halt, it was...",1,NaN,NaN,NaN,NaN
4,1151161501283209222,I’m pretty sure the first four hours back in m...,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
216,1169035797774962688,"I wonder if this will have a chilling effect, ...",0,NaN,NaN,NaN,NaN
217,240984729288065024,school was mentally exhausting today,0,NaN,NaN,NaN,NaN
218,1091698861524037632,dragons-and-bees: thinking about……..her reblog...,0,NaN,NaN,NaN,NaN
219,213343876260433920,"One of the best skins quotes, ""You are a glori...",0,NaN,NaN,NaN,NaN


In [46]:
#########Check the data###############
print("Total Length of the Test.csv File received:",len(X_test))
print("Sarcastic Tweets out of 221 is:",test_data.sarcastic.sum())
print("Non-sarcastic", len(X_test)-test_data.sarcastic.sum())

Total Length of the Test.csv File received: 221
Sarcastic Tweets out of 221 is: 119
Non-sarcastic 102


In [36]:
#######repeating the same step as we did in train to create a torch set###################
test_dataset = Dataset(X_test_tokenized)


In [37]:
###########Here we load the best model that we think was best while training########
###########Loading the checkpoint#############
model_path = "output/checkpoint-500"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

loading configuration file output/checkpoint-500/config.json
Model config BertConfig {
  "_name_or_path": "output/checkpoint-500",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file output/checkpoint-500/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenc

In [38]:
###########Using the trainer function again as we did in the training phase#############
test_trainer = Trainer(model)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [39]:
############Results of predicting###############
raw_pred, label_ids, metr = test_trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 221
  Batch size = 8


In [23]:
raw_pred

array([[ 0.08749842, -0.2744067 ],
       [ 0.4132349 , -0.5957164 ],
       [ 0.49343836, -0.7413304 ],
       [ 0.43276814, -0.6278017 ],
       [ 0.48182133, -0.7510359 ],
       [ 0.03415803, -0.16098931],
       [ 0.5813881 , -0.8566219 ],
       [ 0.38638693, -0.59525496],
       [ 0.2322305 , -0.4183592 ],
       [ 0.521226  , -0.761232  ],
       [ 0.54858965, -0.7887273 ],
       [ 0.4072962 , -0.70761603],
       [-0.37529153,  0.1776417 ],
       [-0.16335975, -0.03514753],
       [ 0.43024564, -0.6518344 ],
       [-0.30464792,  0.17650078],
       [-0.17213564,  0.0282666 ],
       [ 0.6226983 , -0.9176838 ],
       [ 0.5156046 , -0.84291637],
       [ 0.43384582, -0.6799662 ],
       [ 0.5612102 , -0.8036428 ],
       [-0.3612378 ,  0.19408132],
       [ 0.09070577, -0.31995302],
       [ 0.5014735 , -0.76116455],
       [ 0.02862571, -0.09249184],
       [-0.11068401, -0.1077878 ],
       [-0.13752054, -0.04502222],
       [-0.04638946, -0.18711783],
       [ 0.33941597,

In [40]:
#######we now have the ra pred which ahas to be pre-processed with argmax where we already defined###########
y_pred = np.argmax(raw_pred, axis=1)

In [27]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [41]:
from sklearn import metrics
print("Accuracy: %.3f" % metrics.accuracy_score(Y_test,y_pred))
cm=metrics.confusion_matrix(Y_test,y_pred)
print("\nConfusion Matrix:\n",cm)

Accuracy: 0.588

Confusion Matrix:
 [[97  5]
 [86 33]]


In [42]:
#how many predictions are right out of sarcastic tweets
print("Recall: %.3f" % metrics.recall_score(Y_test,y_pred))

Recall: 0.277


In [43]:
#How many postives are truly identified
print("Precision: %.3f" % metrics.precision_score(Y_test,y_pred))

Precision: 0.868


In [44]:
#Takes both sarcastic and non-sarcastic contents
print("F1: %.3f" % metrics.f1_score(Y_test,y_pred))

F1: 0.420
